<a href="https://colab.research.google.com/github/pmontman/M4metaresults/blob/master/nb/tuto_06_panel_mixed_logit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 6: Panel data and mixed logit

We will analyze a marketing dataset, of choice of brand of catsup (a.k.a. ketchup or 'tomato sauce' in Australia).

We have two famous brands of catsup and 3 different package sizes.

A description of the dataset can befound [here](https://www.tandfonline.com/doi/pdf/10.1080/07350015.1994.10524547?casa_token=r4LpjVvgDW4AAAAA:FVG8mEexsQ37tJ2bvk7oxZZ9K_jvvMJ2WxglLzBaHQD0_0REkXmKGsPPxXw_LRGwN3YHY8-L-k8U)

It is takend from the mlogit R package (another good source for datasets).

# Description of the dataset

* **id**: household identifiers,
* **choice**: one of heinz41, heinz32, heinz28, hunts32.
* **disp_x**: is there a display for brand X ?
* **feat_x**: is there a newspaper feature advertisement for brand x?
* **price_x**: price of brand x

---
---

# Preparing the environment
*The preparation and dataset loading code is given to the students*

In [1]:
!pip install biogeme

Load the packages, feel free to change the names.

In [2]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools
import biogeme.distributions as dist

# Load the dataset

In [3]:
path = 'https://raw.githubusercontent.com/pmontman/pub-choicemodels/main/data/catsup.csv'
catsup_pd = pd.read_csv(path)


In this case, notice the id variable, that identifies each household, so we have data from many choice situations for each household. We have also different amount of observations per household.

In [4]:
catsup_pd.head(25)

,id,disp_heinz41,disp_heinz32,disp_heinz28,disp_hunts32,feat_heinz41,feat_heinz32,feat_heinz28,feat_hunts32,price_heinz41,price_heinz32,price_heinz28,price_hunts32,choice
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28
5,1,0,0,0,0,0,0,0,0,5.0,3.0,4.7,3.0,heinz28
6,1,0,0,0,1,0,0,0,1,5.1,3.1,4.6,4.1,heinz28
7,1,0,0,0,0,0,0,0,0,4.6,3.4,4.7,3.1,heinz41
8,1,0,0,0,0,0,0,0,0,5.0,3.4,4.7,3.1,heinz28
9,1,0,0,0,1,0,0,0,0,5.0,3.4,5.0,2.8,heinz28


# Auxiliary function

In [5]:
def qbus_update_globals_bgm(pd_df):
   globals().update(db.Database('tmp_bg_bgm_for_glob', pd_df).variables)

# Data cleaning: Preparing the dataset for Biogeme

Encode the choice variable (a string) into numbers with the `factorize` function.
We take a look at the codetable to know how the numbers are mapped to the alternatives. The order of the codetable indicates the association,
0: heinz28, 1: heinz41 and so on.

In [6]:
catsup_pd['choice'], codetable = catsup_pd['choice'].factorize()

In [7]:
codetable

Index(['heinz28', 'heinz41', 'heinz32', 'hunts32'], dtype='object')

#Capturing agent effect through mixed logit

In [8]:

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setSilent()


We will have to use some additional functionalities of biogeme so the auxiliary functions cannot be used. So we have to create the biogeme database manually.

In [9]:
database = db.Database("catsup", catsup_pd)

This line of code tells biogeme to use the variable id in the dataset as the identifier of the individuals to treat them as panel data (if not, it will take observations as if they were independent).

In [10]:
database.panel("id")

We now declare the coefficients in our model, we will make a simple model,
just the alternative-specif constants and the variables disp, feat and price.

In [11]:
ASC_heinz41 = exp.Beta('ASC_heinz41',0,None,None,0)
ASC_heinz32  = exp.Beta('ASC_heinz32 ',0,None,None,0)
ASC_heinz28 = exp.Beta('ASC_heinz28',0,None,None,1)
ASC_hunts32 = exp.Beta('ASC_hunts32',0,None,None,1)

B_disp = exp.Beta('B_disp',0,None,None,0)
B_feat = exp.Beta('B_feat',0,None,None,0)
B_price = exp.Beta('B_price',0,None,None,0)

Now comes the **important part**, the definition of the random effects in the model!

We want to consider a simple constant agent effect.
Recall the definition:

$$V_{jit} = \beta X_{jit} + \alpha_{ij}$$
and we have to specifiy the distribition of the $\alpha$. In this case, we will use the normal, so $\alpha_{ij} \in N(\mu_j, \sigma_j^2)$.
The ingredients are:
 1) Probability distrbitution: We set it ourselves, the normal (we could have set others, such as the uniform, lognormal and so on.
 2) Parameters of the probability distribution (the $\mu$ and $\sigma$): They will be estimated from the data.

 If we think for a moment, the $\mu$ of this distribution will also depend on the values of the alternative-specific constants (ASC), since changing $mu$ essential mean adding a constant to all values coming for that distribution. We can think that the mean of that distribution will be 'absorbed' by the ASCs.
Again this is just another convention!

What we end up doing in biogeme is that we will only declare the parameter for the standard deviation, so each normal distribution will be mean 0 and std.dev to determine from the data. 


In terms of code, we delcare the std.devs (the $\sigma_j$} just as any other parameter. We have to set on of them to 0 to act as reference (remember that changes in scale do not affect utility).

In [12]:

SIGMA_heinz41 = exp.Beta('SIGMA_heinz41',0,0,None,0)
SIGMA_heinz32 = exp.Beta('SIGMA_heinz32',0,0,None, 1)
SIGMA_heinz28 = exp.Beta('SIGMA_heinz28',0,0,None,0)
SIGMA_hunts32 = exp.Beta('SIGMA_hunts32',0,0,None,0)


And the following code is how we tell biogeme that the parameters are random.
The `EC_`s are the agent effects. The key function that indicate randomness is the biogeme function `exp.bioDraws`, that indicates that they are random, drawn from a probability distribution. The second argument specifies the distribution, some possible values are `'NORMAL'` `'UNIFORMSYM'`.

In [13]:

# Define random parameters, normally distributed across individuals,
# designed to be used for Monte-Carlo simulation
EC_heinz41 = SIGMA_heinz41 * exp.bioDraws('EC_heinz41','NORMAL')
EC_heinz32 = SIGMA_heinz32 * exp.bioDraws('EC_heinz32','NORMAL')
EC_heinz28 = SIGMA_heinz28 * exp.bioDraws('EC_heinz28','NORMAL')
EC_hunts32 = SIGMA_hunts32 * exp.bioDraws('EC_hunts32','NORMAL')

An this is how the specification of the utility functions looks like in the end.
The Betas (disp, feat and price) and ASCsare fixed effect, while the EC are random.

In [14]:
globals().update(database.variables)
# Definition of the utility functions
V_heinz41 = ASC_heinz41 + B_disp *disp_heinz41 + B_feat * feat_heinz41 + B_price * price_heinz41 + EC_heinz41
V_heinz32 = ASC_heinz32 + B_disp *disp_heinz32 + B_feat * feat_heinz32 + B_price * price_heinz32 + EC_heinz32
V_heinz28 = ASC_heinz28 + B_disp *disp_heinz28 + B_feat * feat_heinz28 + B_price * price_heinz28 + EC_heinz28
V_hunts32 = ASC_hunts32 + B_disp *disp_hunts32 + B_feat * feat_hunts32 + B_price * price_hunts32 + EC_hunts32

We create the dictionary that maps to the alternatives. **Remember to be careful here.** The numbers should match the alternatives as we indicated by the factorize() transformation at the beginning of the notebook.
Availabilities are not considered here, we set them to 1.
Finaly, we specify the logit model, as usual.
These steps are common to the multinomial logit.

In [15]:
# Associate utility functions with the numbering of alternatives
V = {0: V_heinz28,
     1: V_heinz41,
     2: V_heinz32,
     3: V_hunts32}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

# Conditional to the random variables, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = models.logit(V,av, choice)


The difference from the usual declaration of the MNL comes now.

We have to do two new steps:

1. Tell biogeme to consider the panel nature of the data.
2. Tell biogeme to calculate the choice probabilities by simulation. This is how we deal with the random parameters. We simulate for the distribution, and the we calculate the likelihood for that distribution.

Step 1: we can do it by modifying the model with the expression `exp.PanelLikelihoodTrajectory`.

In [16]:
condprobIndiv = exp.PanelLikelihoodTrajectory(obsprob)

And Step 2 we take the model and the modifyi it by the expresion `exp.MonteCarlo`. The final log it to take the loglikelihood.

In [17]:
logprob = exp.log(exp.MonteCarlo(condprobIndiv))

We we are using simulation, we have to tell biogeme how many draws from the distribution are we going to generate. The more draws, the more accurate estimation, but it is compuationally costly.

We also set up a seed, so we can get the same results if the run the notebook again (setting up a seed is a good habit in general)

In [18]:

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob,numberOfDraws=250, seed=1)




Estimation and results as usual.

In [19]:

# Estimate the parameters. 
results = biogeme.estimate()

We take a look at the results,

In [20]:
results.getEstimatedParameters()

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz32,-0.729645,0.0,8.312623e-02,-8.777548,0.000000e+00,0.151278,-4.823198,1.412747e-06
ASC_heinz41,-1.490036,0.0,1.517855e-01,-9.816716,0.000000e+00,0.136599,-10.908122,0.000000e+00
B_disp,1.043355,0.0,1.108189e-01,9.414955,0.000000e+00,0.128278,8.133542,4.440892e-16
B_feat,0.986969,0.0,1.237311e-01,7.976729,1.554312e-15,0.135843,7.265501,3.717027e-13
B_price,-1.272207,0.0,6.252270e-02,-20.347917,0.000000e+00,0.089295,-14.247292,0.000000e+00
SIGMA_heinz28,0.000000,1.0,1.797693e+308,0.000000,1.000000e+00,0.004437,0.000000,1.000000e+00
SIGMA_heinz41,1.235048,0.0,1.412200e-01,8.745558,0.000000e+00,0.165150,7.478355,7.527312e-14
SIGMA_hunts32,4.847132,0.0,4.066168e-01,11.920637,0.000000e+00,0.418897,11.571186,0.000000e+00


Interestingly, the simulation of panel data is not implemented! No problem
we will still be able to do simulations, by setting up a scenario that does not consider the panel data.

In [21]:
#biogeme.simulate(results.getBetaValues())

# Compare Panel vs Not using the panel information

We will just compare the results that we get if the just ignore the panel information. We can recreate this by not using `exp.PanelLikelihoodTrajectory` in the model.

In math, this would be the specification:

$$V_{jit} = \beta X_{jit} + \alpha_{ij \color{red}{t}}$$

Os opposed to the panel specification
$$V_{jit} = \beta X_{jit} + \alpha_{ij}$$

Notice the difference subindex $t$.

In [22]:
database_nonpanel = db.Database("catsup", catsup_pd)


# We integrate over the random variables using Monte-Carlo
logprob_nonpanel = exp.log(exp.MonteCarlo(obsprob))


# Create the Biogeme object
biogeme_nonpanel  = bio.BIOGEME(database_nonpanel,logprob_nonpanel,numberOfDraws=250, seed=1)


# Estimate the parameters. 
results_nonpanel = biogeme_nonpanel.estimate()


In [23]:

results_nonpanel.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz32,1.635916,0.094591,17.294557,0.000000e+00,0.092826,17.623540,0.000000e+00
ASC_heinz41,0.974421,0.369801,2.634984,8.414124e-03,0.368591,2.643639,8.202004e-03
B_disp,1.467588,0.194329,7.552096,4.285461e-14,0.214648,6.837200,8.075540e-12
B_feat,1.728661,0.236286,7.315962,2.555733e-13,0.262917,6.574921,4.867884e-11
B_price,-2.357343,0.156495,-15.063384,0.000000e+00,0.159830,-14.749045,0.000000e+00
SIGMA_heinz28,6.755450,0.456698,14.791943,0.000000e+00,0.456737,14.790665,0.000000e+00
SIGMA_heinz41,2.473500,0.399854,6.186002,6.170917e-10,0.415264,5.956452,2.577732e-09
SIGMA_hunts32,0.205258,0.318454,0.644543,5.192232e-01,0.111694,1.837673,6.611060e-02


Simulation of the choice probabilities is similar to the MNL, we just have
to 

In [24]:
tgt_nonpanel = {
    0: exp.MonteCarlo(models.logit(V, av, 0)),
    1: exp.MonteCarlo(models.logit(V, av, 1)),
    2: exp.MonteCarlo(models.logit(V, av, 2)),
    3: exp.MonteCarlo(models.logit(V, av, 3))
    }


In [25]:

sim_nonpanel = bio.BIOGEME(database_nonpanel, tgt_nonpanel, numberOfDraws=250, seed=1)
preds = sim_nonpanel.simulate(theBetaValues=results_nonpanel.getBetaValues())
preds

,0,1,2,3
0,0.221586,0.125526,0.466292,0.186596
1,0.283891,0.242898,0.408833,0.064378
2,0.093050,0.006497,0.900311,0.000142
3,0.184734,0.122233,0.493843,0.199190
4,0.311898,0.042161,0.644101,0.001840
...,...,...,...,...
2793,0.221344,0.263944,0.368983,0.145729
2794,0.315515,0.083315,0.479933,0.121236
2795,0.162401,0.016769,0.067577,0.753253
2796,0.228224,0.059167,0.276098,0.436511


In [26]:
sim_panel = bio.BIOGEME(database_nonpanel, tgt_nonpanel, numberOfDraws=250, seed=1)
preds = sim_panel.simulate(theBetaValues=results.getBetaValues())
preds

,0,1,2,3
0,0.087785,0.067616,0.285303,0.559296
1,0.159617,0.112806,0.241801,0.485777
2,0.036349,0.014350,0.680025,0.269275
3,0.087996,0.064651,0.285991,0.561363
4,0.263782,0.037087,0.362868,0.336263
...,...,...,...,...
2793,0.121595,0.127612,0.237574,0.513220
2794,0.221157,0.036320,0.201408,0.541116
2795,0.063142,0.021121,0.180699,0.735038
2796,0.103882,0.040646,0.261773,0.593699


#Compare to the Multinomial Logit (fixed effects) without agent effect

Just remove the random effect when specifying the utility functions.

In [27]:
V_heinz41_mnl = ASC_heinz41 + B_disp *disp_heinz41 + B_feat * feat_heinz41 + B_price * price_heinz41 #+ EC_heinz41
V_heinz32_mnl = ASC_heinz32 + B_disp *disp_heinz32 + B_feat * feat_heinz32 + B_price * price_heinz32 #+ EC_heinz32
V_heinz28_mnl = ASC_heinz28 + B_disp *disp_heinz28 + B_feat * feat_heinz28 + B_price * price_heinz28 #+ EC_heinz28
V_hunts32_mnl = ASC_hunts32 + B_disp *disp_hunts32 + B_feat * feat_hunts32 + B_price * price_hunts32 #+ EC_hunts32

In [28]:
V_mnl = {0: V_heinz28_mnl,
     1: V_heinz41_mnl,
     2: V_heinz32_mnl,
     3: V_hunts32_mnl}

In [29]:
 logprob = models.loglogit (V_mnl , av , choice )
 bgm_model = bio.BIOGEME ( database_nonpanel, logprob )
results_mnl = bgm_model.estimate()

In [30]:
results_mnl.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz32,0.648626,0.045939,14.119377,0.000000e+00,0.041539,15.614902,0.000000e+00
ASC_heinz41,-0.634554,0.088652,-7.157828,8.195666e-13,0.092739,-6.842360,7.789991e-12
B_disp,1.364851,0.091346,14.941472,0.000000e+00,0.097956,13.933276,0.000000e+00
B_feat,0.939845,0.101568,9.253366,0.000000e+00,0.101292,9.278530,0.000000e+00
B_price,-0.519478,0.038763,-13.401433,0.000000e+00,0.040612,-12.791336,0.000000e+00


In [31]:
tgt_mnl = {
    0: (models.logit(V_mnl, av, 0)),
    1: (models.logit(V_mnl, av, 1)),
    2: (models.logit(V_mnl, av, 2)),
    3: (models.logit(V_mnl, av, 3))
    }

sim_mnl = bio.BIOGEME(database_nonpanel, tgt_mnl)
preds_mnl = sim_mnl.simulate(theBetaValues=results_mnl.getBetaValues())
preds_mnl

,0,1,2,3
0,0.118467,0.085779,0.493974,0.301780
1,0.158921,0.115070,0.485202,0.240806
2,0.058798,0.031173,0.857034,0.052995
3,0.118467,0.085779,0.493974,0.301780
4,0.305327,0.063243,0.523914,0.107516
...,...,...,...,...
2793,0.136931,0.115868,0.463834,0.283367
2794,0.192460,0.067340,0.477352,0.262848
2795,0.050414,0.026728,0.199569,0.723289
2796,0.118707,0.062935,0.446128,0.372230


---
---

# Exercise: Caaturing dynamics: Add last choice as additional variable (assume that data was observed in order), add it as fixed parameter and  estimate a mixed logit.
Basically we add a new variable and repeat the process for estimating the mixed logit.

The first step is given to us: In the following cells we are going to create a new dataset that has an additional covariate representing the alternative that was chosen before each choice situation.


In [32]:
catsup_past = catsup_pd.copy()

This functions takes a column, removes the last observation and adds a -1 at the begginning. This is how we create the lagged variable.

In [43]:
def last_choice(x):
  return pd.Series([-1]).append(x[:-1])

We apply the function `last_choice` to the dataset, but we group the dataset by the id of the household.

In [54]:
lchoice =  catsup_past.groupby('id')['choice'].apply(last_choice).reset_index()#.head(25)
catsup_past['last_choice'] = lchoice['choice']
catsup_past.head(17)

,id,disp_heinz41,disp_heinz32,disp_heinz28,disp_hunts32,feat_heinz41,feat_heinz32,feat_heinz28,feat_hunts32,price_heinz41,price_heinz32,price_heinz28,price_hunts32,choice,_biogroups,last_choice
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,0,1,-1
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,0,1,0
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,0,1,0
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,0,1,0
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,0,1,0
5,1,0,0,0,0,0,0,0,0,5.0,3.0,4.7,3.0,0,1,0
6,1,0,0,0,1,0,0,0,1,5.1,3.1,4.6,4.1,0,1,0
7,1,0,0,0,0,0,0,0,0,4.6,3.4,4.7,3.1,1,1,0
8,1,0,0,0,0,0,0,0,0,5.0,3.4,4.7,3.1,0,1,1
9,1,0,0,0,1,0,0,0,0,5.0,3.4,5.0,2.8,0,1,0


But now it is up to you how the new variable is added to the model!
Transformations? Dummy encoding? Per-alternative parameters?.